<a href="https://colab.research.google.com/github/Sa1syo/NLTK/blob/main/IRNLP2019_Ex11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 11. Information Retrieval 1

Exercise 11 and 12 are original contents of IR&NLP in Univ. of Aizu.
Please read carefully in this page and try their source code.


本演習では、検索エンジンを作るための簡単な例として、今まで勉強してきたNLTKの扱い方を駆使して、簡単な情報検索システムを作成しましょう。  
In this exercise, we try to make a kind of document search engine utilize NLTK as to construct search engine.

検索対象は世界的なベストセラーの聖書(Bible-KJV)の中に含まれる66の書物、計1183章、31173節あります。  
Target document is Bible-KJV (it has 66 books, 1183 chapters and 31173 sections in total) which is a best-seller book in world wide.

ここでは、書物毎, 行（凡そ節ごと、但し連接している場合は1つの行とみなす）毎のWord Count及びTF\*IDFを学びます。  
We study about how to index all the documents using word count and TF\*IDF in each books and lines (or chapters).


## 11.1. Preprocessing
- 対象とするデータ: Gutenberg Corpusから, Bible-KJV  
 Target Data: Bible-KJV from Gutenberg Corpus. (Please remember how to load a book from Gutenberg Corpus in Ex.2)
- UTF-8で記述されているRawデータから, Parsing, Stemmingを経て、Stemmingの掛かったTextオブジェクトへの変換を行う。  
 Convert the text object wihch processed parsing and lemmatizing from raw data which is UTF-8.
  - http://www.gutenberg.org/cache/epub/10/pg10.txt

In AY2020, urllib is not able to move enough, then we will use "requests" module.  
If you need some information about urllib, please go to Chapter 3, to download from gutenberg.org

In [ ]:
import requests
r = requests.get('http://www.gutenberg.org/cache/epub/10/pg10.txt')
raw = r.text

In [ ]:
# 平文1000文字分を表示
# Print 1000 characters
print(raw[:1000])

﻿The Project Gutenberg eBook of The King James Bible

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: The King James Bible

Release Date: August, 1989 [eBook #10]
[Most recently updated: December 20, 2021]

Language: English


*** START OF THE PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***




The Old Testament of the King James Version of the Bible
The First Book of Moses: Called Genesis
The Second Book of Moses: Called Exodus
The Third Book of Moses: Called Leviticus
The Fourth Book of Moses: Called Numbers
The Fifth Book of Moses: Called Deuteronomy
The Boo


### 11.1.1. Direct search in a document

NLTK Book chapter 3によれば、Regular Expressionを用いて直接的にString Matchingを行うことが出来る。  
Pythonに含まれるRegular Expressionに対応するMethodであるfind()やrfind()を用いると、直接Raw Textから特定の語やPatternを見つけることが出来る。 

According to NLTK Book chapter 3, String Matching can be performed directly using Regular Expression.  
If you use find() or rfind() which are methods corresponding to regular expression included in Python, you can find specific words and patterns directly from raw Text.

In [ ]:
# 原始的な検索: findやrfindを用いると可能。取得した場所から100語を抽出
a = raw.find("Jesus Christ")
print(a)
print(raw.rfind("Jesus Christ"))
print(raw[a:a+100])

3413560
4435139
Jesus Christ, the son of David, the
son of Abraham.

1:2 Abraham begat Isaac; and Isaac begat Jac


### 11.1.2. Tokenize

ここでは、章・行・単語の分割を行う。また、どの行が章に、どの単語がどの行に含まれているのかを記述する。  
In this section, we try to divide into chapters, lines, and words from string sequence. We also extract the information which lines are included in the chapter and which words are included in which lines.

In [ ]:
#from __future__ import division, print_function # Python 2 users only
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# 多数改行する部分を捉えて、文としてピリオドを付与する
# Insert period as sentence to find line feed continuity. 
raw2 = re.sub(r'\r\n\r\n(\r\n)+', ".\r\n", raw)
raw2 = re.sub(r'\.\.', ".", raw2) # 上記作業でできた2重ピリオドを削除 delete double period

# 入力したものを、文や単語ごとにトークン化
# Tokenize processed string
sentences = sent_tokenize(raw2)
tokens = word_tokenize(raw2)

print("Num. of Sentences: " + str(len(sentences)))
print("Num. of Words: " + str(len(tokens)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Num. of Sentences: 30002
Num. of Words: 952530


In [ ]:
# 20行分を表示。Print 20 lines
for i in range(101,120):
    print(sentences[i])
    print("-----")

4:11 And now art thou cursed from the earth, which hath opened her
mouth to receive thy brother’s blood from thy hand; 4:12 When thou
tillest the ground, it shall not henceforth yield unto thee her
strength; a fugitive and a vagabond shalt thou be in the earth.
-----
4:13 And Cain said unto the LORD, My punishment is greater than I can
bear.
-----
4:14 Behold, thou hast driven me out this day from the face of the
earth; and from thy face shall I be hid; and I shall be a fugitive and
a vagabond in the earth; and it shall come to pass, that every one
that findeth me shall slay me.
-----
4:15 And the LORD said unto him, Therefore whosoever slayeth Cain,
vengeance shall be taken on him sevenfold.
-----
And the LORD set a mark
upon Cain, lest any finding him should kill him.
-----
4:16 And Cain went out from the presence of the LORD, and dwelt in the
land of Nod, on the east of Eden.
-----
4:17 And Cain knew his wife; and she conceived, and bare Enoch: and he
builded a city, and called the 

In [ ]:
# KJVの内容の最後には、下記のようなパターンがあるので、これを種にして、最終行を得る。
# Extract last line using the code of end pattern in Gutenberg Corpus.
finreg = re.compile(r'END OF THE PROJECT')
endline = [(s1-1, sentences[s1-1]) for s1, s2 in enumerate(sentences) if finreg.search(s2)]
print(endline)

[(29886, 'Amen.')]


実際の文書をセグメント化しようとする場合、文書の事前知識を用いるケースが多い。  
In you want to segment real documents, we use pre knowledge of this document in many case.

聖書では、最初の節は『1章1節』であるので、この場合では 1:1 というパターンが出る。  
これを捉えて、タイトル行の行番号を得る。
In the Bible-KJV dataset has the pattern '^\\:1 ' after the title line. We can use this information to extract book title.

In [ ]:
#書物の情報の抽出: 聖書は各章ほぼ必ず1:1から始まるので、マッチングした行の1つ上を抽出
# Extracting Book Information: The Bible almost always starts at 1: 1 so extract one line above the matched line
regexp = re.compile(r'^1\:1 ')
finreg = re.compile(r'^End of the Project')
titlelist = [(s1-1, sentences[s1-1]) for s1, s2 in enumerate(sentences) if regexp.search(s2)]
print(len(titlelist))
for t in titlelist:
    print(t)
    print('-----')

66
(7, 'The First Book of Moses:  Called Genesis.')
-----
(1475, 'The Second Book of Moses:  Called Exodus.')
-----
(2587, 'The Third Book of Moses:  Called Leviticus.')
-----
(3252, 'The Fourth Book of Moses:  Called Numbers.')
-----
(4244, 'The Fifth Book of Moses:  Called Deuteronomy.')
-----
(4975, 'The Book of Joshua.')
-----
(5468, 'The Book of Judges.')
-----
(6176, 'The Book of Ruth.')
-----
(6280, 'The First Book of Samuel\r\n\r\nOtherwise Called:\r\n\r\nThe First Book of the Kings.')
-----
(7294, 'The Second Book of Samuel\r\n\r\nOtherwise Called:\r\n\r\nThe Second Book of the Kings.')
-----
(8133, 'The First Book of the Kings\r\n\r\nCommonly Called:\r\n\r\nThe Third Book of the Kings.')
-----
(8961, 'The Second Book of the Kings\r\n\r\nCommonly Called:\r\n\r\nThe Fourth Book of the Kings.')
-----
(9850, 'The First Book of the Chronicles.')
-----
(10616, 'The Second Book of the Chronicles.')
-----
(11444, 'Ezra.')
-----
(11672, 'The Book of Nehemiah.')
-----
(12027, 'The Book

In [ ]:
# 書物内のデータを再分割。行分割を行っていたので、上記の書物情報の行番号を頼りに、
# 書物のタイトルを除いた本文を一つのリストにしている。
# Subdivide the data in the book. 
# Because the line is divided, the text excluding the title of the book is made into one list, 
# relying on the line number of the book information.
books = []
prev = 0
for i, title in enumerate(titlelist):
    book = []
    if i == 0:
        prev = title[0]
        continue
    for j in range(title[0]-prev-1):
        book += word_tokenize(sentences[prev+j+1])
    books.append(book)
    prev = title[0]
book = []
for i in range(endline[0][0]-prev):
    book += word_tokenize(sentences[title[0]+i+1])
books.append(book)

節の分離: 行内に入り込んでしまっているケースが多いので、一単語ずつ丁寧にSearchする必要がある。  
Separation of clauses: Patterns indicating clauses buried in the line sometime, so it is necessary to find one by one carefully in each word.

In [ ]:
# 節に関してはちゃんと節毎に分かれていない。
bookplace = []
verses = []
verse = []
place = 0
re_sever = re.compile(r'^[1-9][0-9]*\:[1-9][0-9]*')
for j,book in enumerate(books):
    bookplace.append(place)
    for i, s in enumerate(book):
        if re_sever.search(s):
            if verse != []:
                verses.append(verse)
                place += 1
            verse = []
            verse.append(s)
        else:
            verse.append(s)
    verses.append(verse)
    place += 1
    verse = []


In [ ]:
# 節数の確認, Check the number of verses, but this is less than true.
print(len(verses))

In [ ]:
# Versesの中の書物の先頭の場所, Check the number of position which is indicates start of each verse
print(bookplace)

### 11.1.3. Stemming, Lemmatizing

(★ Assignment Remark):

検索のための単語をそろえるために、StemmingやLemmatizingを行う。  
To standardize words for search, we shuold do stemming and lemmatizing.   

 - Stemmingは基本的に語の時制等『接頭語・接尾語』などを削るために行う。  
 Stemming is able to eliminate prefix and postfix to make standardization of a meaning of words.
 - LemmatizingはStemmingで処理出来ない変化形の語ををWordnetなどの辞書を用いて正規化するために行う。  
 Lemmatizing is able to trace the changing of words such as eat to ate which is not able to transform by Stemming.

ここでは、Porter Stemmerを用いたStemmingの例として、Chapter 3.3で行ったIndexTextクラスを用いる。さらに、Lemmatizerを用いたLemmatizingの例として、IndexTextクラスを修正したLemmatizedTextクラスを作成する。  
Here, we construct IndexText class which utilize Porter stemmer for stemming, and LemmatizedText class which utilize Wordnet lemmatizer.

In [ ]:
# Text Normalization: Ch.3で行ったクラスを利用
class IndexedText(object):

    def __init__(self, stemmer, text):
        self._text = text
        self._stemmer = stemmer
        self._index = nltk.Index((self._stem(word), i)
                                 for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._stem(word)
        wc = int(width/4)                # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            ldisplay = '{:>{width}}'.format(lcontext[-width:], width=width)
            rdisplay = '{:{width}}'.format(rcontext[:width], width=width)
            print(ldisplay, rdisplay)

    def _stem(self, word):
        return self._stemmer.stem(word).lower()

In [ ]:
# Stemming: Porter Stemmerを利用
porter = nltk.PorterStemmer()
text = IndexedText(porter, tokens)

In [ ]:
# Concordanceの利用: 一種のSearch Engineと言える。
text.concordance('say')

In [ ]:
# WordNetを用いたLemmatizerの例 (Chapter 3.4.)
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()
tkn = [wnl.lemmatize(t) for t in tokens]

In [ ]:
# Stemmer Indexの改善: Wordnet Lemmatizerを用いた場合
class LemmatizedText(object):

    def __init__(self, lemmatizer, text):
        self._text = text
        self._lemmatizer = lemmatizer
        self._index = nltk.Index((self._lemmatize(word), i)
                                 for (i, word) in enumerate(text))

    def concordance(self, word, width=40):
        key = self._lemmatize(word)
        wc = int(width/4)                # words of context
        for i in self._index[key]:
            lcontext = ' '.join(self._text[i-wc:i])
            rcontext = ' '.join(self._text[i:i+wc])
            ldisplay = '{:>{width}}'.format(lcontext[-width:], width=width)
            rdisplay = '{:{width}}'.format(rcontext[:width], width=width)
            print(ldisplay, rdisplay)

    def _lemmatize(self, word):
        return self._lemmatizer.lemmatize(word).lower()

In [ ]:
# Lemmatizing: Wordnet Lemmatizerを利用
text2 = LemmatizedText(wnl, tokens)

*Index化の結果*
Result of indexing

In [ ]:
print(list( (v, text2._index[v]) for i,v in enumerate(text2._index) if i > 140 and i < 145 ) )

In [ ]:
# Concordanceの利用: Stemmingと違い、Sayingのサポートがない
text2.concordance('say')

Lemmatizerでは、Stemmingのようなことは成されないが、一方で、名詞が壊れる等の問題は少ない。

In [ ]:
# Stemming, Lemmatizingを行った後、単語リストとして使用するためのリスト化操作
keys = text2._index.keys()
# Python3は以下も行う
keys = [*keys]
print(keys[:100])

### 11.1.4 POS Tagging and Extract Feature

(★ Assignment Remark)

Lecture 6において、Featureを取り出すには、有用なFeatureを考察する必要がある。  
From the lecture 6, if we want to extract feature from documents, we should consider what is the effective feature.

特に、検索では、全文を検索するのは非常に時間が掛かるために、限られた特徴量に絞って検索を行うことが多い。  
Especiall in a search, we often perform a search with a limited amount of features since it takes a very long time to search the entire text.

本パートでは、POSタグ付けを行い、名詞、動詞、形容詞のうち、全体的に出現頻度の高い語を特徴量として扱う。  
In this part, we perform POS tagging on words, and treat words with high overall appearance frequency as features of nouns, verbs, and adjectives.

In [ ]:
#Feature Vectorの構築: WordlistからStopwordsを省く。基本的に名詞や動詞、形容詞を利用。
# NLTK内部に入っているPOS TAGを利用。文脈を見ているわけではないのであまり精度は良くない
nltk.download('averaged_perceptron_tagger')
wordtags = nltk.pos_tag(keys)

In [ ]:
nltk.FreqDist(tag for (word, tag) in wordtags)

In [ ]:
tkns = nltk.FreqDist(word for word in tkn)
tknlst = tkns.most_common(5000)
feat = list(w.lower() for (w,c) in tknlst)
# for listing names
feat2 = list(w for w,c in tknlst)

In [ ]:
# Stopwordを省いた上で、今回は名詞と動詞のみで構成する要素を抽出
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
features = [(word , tag) for (word , tag) in wordtags if word not in stopWords and word in feat 
            if tag == 'NN' or tag == 'NNP' or tag =='PP' or tag == 'PRP' or tag =='VBD' or tag == 'VBG'
            or tag == 'VBP' or tag == 'VBN' or tag == 'JJ' or tag == 'JJS' or tag == 'JJR']

In [ ]:
nltk.FreqDist(tag for (word, tag) in features)

In [ ]:
# [6]で書物情報を抽出しているので、書物毎に出てくる単語をLemmatizeしておく。
# 各書物毎のLematizeされた単語リストを形成
lemmabooks = []
for book in books:
    lemmabook = []
    for w in book:
        lemmabook.append(wnl.lemmatize(w).lower())
    lemmabooks.append(lemmabook)
collection = nltk.TextCollection(lemmabooks)

In [ ]:
# 同様に、節の区切りもLemmatizeしておくことにする。
lemmaverses = []
for verse in verses:
    lemmaverse = []
    for w in verse:
        lemmaverse.append(wnl.lemmatize(w).lower())
    lemmaverses.append(lemmaverse)
collection2 = nltk.TextCollection(lemmaverses)

In [ ]:
# 特徴的な語彙をそろえるために、出現上位3000語の単語のうち、featureで指定した名詞・動詞・形容詞の単語リストを作成
flist = list ( v.lower() for (v,k) in features)
flist.sort()
print(flist[:100])

In [ ]:
# For listing name and place
tmplist = list( v for v in feat2 if v[0].isupper())
nlist = list( v for v in tmplist if v.lower() not in feat2 and (v.lower(),'NN') in features or (v.lower(), 'NNP') in features)
print(len(nlist))
print(nlist)

# Exercise Attendance

Please make concordance of 3 person's name which select from in previous list (which includes some noise word).  
Also, please find a person's name which has less than 10 times appear in the Bible.

Hint: text.concordance('David')

## 11.2. Feature Vector Construction

検索対象となる特徴ベクトルやIndex化を行う。  
In this section, we make index of feature vector.

ここでのIndexは、
- 単語頻度リスト
- TF・IDFベクトル

である。

In here, the index data is:
 - word count list
 - TF\*IDF vector
 
ここでのTF\*IDFは、単純な掛け算によるTF\*IDFで、対数TF\*IDFではないです。
This TF\*IDF is naive multiplication not log-scale TF\*IDF.

なお、データサイズが大きいので、この部分は非常に時間が掛かるので注意してください。  
Please notice about this process needs to take so much time by datasize.

In [ ]:
# TFIDFの計算。各文書毎に計算を行う。なお、collectionで計算しているので、Featureのみではなく全単語で計算している。
verselist = []
for i, verse in enumerate(lemmaverses):
    fv = []
    wc = []
    for term in flist:
        fv.append(collection2.tf_idf(term, verse))
        wc.append(verse.count(term))
    verselist.append((i, verse[0], fv, wc))

In [ ]:
chaplist = []
for i, doc in enumerate(lemmabooks):
    fv = []
    wc = []
    for term in flist:
        fv.append(collection.tf_idf(term, doc))
        wc.append(doc.count(term))
    chaplist.append((i, titlelist[i][1], fv, wc))

In [ ]:
# 『創世記 1;1』のTFIDFベクトル
print(verselist[0])

In [ ]:
# 『創世記』のTFIDFベクトル
print(chaplist[0])

In [ ]:
# 『ヨハネの黙示録』のTFIDFベクトル
print(chaplist[65])

## 11.3. Retrieval
(★ Assignment Remark)  
Index化が終われば、Indexを走査することで、情報を検索することができる。  
After indexing, you can retrieve relative verse with your query.

幾つかの検索モデルで試してみましょう  
Please check some retrieval model
 - Boolean Model
 - Extended Boolean Model (Fuzzy Model)
 - Vector Space Model

Query List:
- 検索: 『 Is Jesus Christ the son of GOD? 』
- 検索; 『 Can Jesus Christ serves us from our sin by his cross and his grace?』

In [ ]:
# Extended Boolean Model: 単語包含数のMAX検索
def extboolsearch(query, datas, datalist, taglist, lemmatizer):
    tokens = word_tokenize(query)
    lemmas = list( lemmatizer.lemmatize(w.lower()) for w in tokens)
    points = []
    for i, v in enumerate(datalist):
        k = 0
        for word in set(lemmas):
            if word in flist:
                k = max(k, v[2][taglist.index(word)])
        points.append((k,i))
    points.sort(reverse=True)
    for k, i in points[:10]:
        print((k,datas[i]))

In [ ]:
extboolsearch(query1, verses, verselist, flist, wnl)

In [ ]:
extboolsearch(query2, verses, verselist, flist, wnl)

In [ ]:
# Vector Space Model with Cosine Collelation: TF・IDFとQueryの生起確率との内積
def cosinesearch(query, datas, datalist, taglist, lemmatizer, nums):
    tokens = word_tokenize(query)
    lemmas = list( lemmatizer.lemmatize(w.lower()) for w in tokens if w.lower() in flist)
    setlem = set(lemmas)
    points = []
    lenlem = 0
    for s in setlem:
        lenlem = lemmas.count(s)
    for i, v in enumerate(datalist):
        lenv = 0
        for vt in v[2]:
            lenv += vt**2
        k = 0
        for word in set(lemmas):
            if word in flist:
                k += 2 *  v[2][taglist.index(word)]*lemmas.count(word)
        k = k / (lenlem + lenv)
        points.append((k,i))
    points.sort(reverse=True)
    for k, i in points[:nums]:
        print((k, datas[i]))

In [ ]:
cosinesearch(query1, verses, verselist, flist, wnl, 10)

In [ ]:
cosinesearch(query2, verses, verselist, flist, wnl,10)

In [ ]:
cosinesearch(query1, books, chaplist, flist, wnl,1)

In [ ]:
cosinesearch(query2, books, chaplist, flist, wnl,1)